In [ ]:
# Setup
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import json
import requests  # For API calls
from typing import Dict, Any, List

print("✅ API integration libraries loaded!")

---

## 1. REST API Overview

PoliSim provides a REST API for programmatic access:

```
Base URL: http://localhost:8000/api/v1

Endpoints:
  GET  /policies           - List available policies
  GET  /policies/{id}      - Get policy details
  POST /simulate           - Run simulation
  GET  /simulate/{job_id}  - Get simulation results
  GET  /health             - Health check
```

### Authentication
API uses JWT tokens for authentication.

```python
headers = {
    'Authorization': 'Bearer <your-token>',
    'Content-Type': 'application/json'
}
```

In [ ]:
# API Client Class
class PoliSimClient:
    """Simple client for PoliSim API."""
    
    def __init__(self, base_url: str = "http://localhost:8000", token: str = None):
        self.base_url = base_url
        self.token = token
        self.session = requests.Session()
        if token:
            self.session.headers['Authorization'] = f'Bearer {token}'
        self.session.headers['Content-Type'] = 'application/json'
    
    def health_check(self) -> Dict[str, Any]:
        """Check API health."""
        try:
            resp = self.session.get(f"{self.base_url}/health", timeout=5)
            return resp.json() if resp.ok else {'status': 'error', 'code': resp.status_code}
        except requests.exceptions.ConnectionError:
            return {'status': 'offline', 'message': 'API server not running'}
    
    def list_policies(self) -> List[Dict]:
        """List available policies."""
        resp = self.session.get(f"{self.base_url}/api/v1/policies")
        return resp.json() if resp.ok else []
    
    def get_policy(self, policy_id: str) -> Dict[str, Any]:
        """Get specific policy details."""
        resp = self.session.get(f"{self.base_url}/api/v1/policies/{policy_id}")
        return resp.json() if resp.ok else {}
    
    def run_simulation(self, config: Dict[str, Any]) -> Dict[str, Any]:
        """Run a simulation."""
        resp = self.session.post(
            f"{self.base_url}/api/v1/simulate",
            json=config
        )
        return resp.json() if resp.ok else {'error': resp.text}

print("✅ PoliSimClient class defined!")
print("\n📋 Available methods:")
print("   • health_check()   - Check API status")
print("   • list_policies()  - Get all policies")
print("   • get_policy(id)   - Get policy details")
print("   • run_simulation() - Execute simulation")

---

## 2. Offline Mode (Direct Imports)

If the API server isn't running, you can use direct imports:

In [ ]:
# Direct import mode (no API server needed)
from core.simulation import run_comprehensive_simulation
from core.policies import PolicyType, get_policy_config
from core.combined_outlook import CombinedFiscalOutlookModel

print("✅ Direct import mode available!")
print("\n💡 Use this when API server isn't running.")

# Example: Run simulation directly
print("\n🔄 Running direct simulation...")

model = CombinedFiscalOutlookModel()
baseline = model.project_outlook(2024, 2034)

print(f"\n📊 Direct Results (2024-2034):")
print(f"   • Final debt: ${baseline['total_debt'][-1]/1e12:.2f}T")
print(f"   • Final deficit: ${baseline['annual_deficit'][-1]/1e9:.0f}B")

---

## 3. Batch Processing

Run multiple scenarios efficiently:

In [ ]:
# Batch simulation runner
def batch_simulate(scenarios: List[Dict], years: int = 10) -> List[Dict]:
    """Run multiple simulation scenarios."""
    results = []
    
    for scenario in scenarios:
        print(f"🔄 Running: {scenario['name']}...")
        
        # Configure model with scenario parameters
        model = CombinedFiscalOutlookModel()
        
        # Apply scenario adjustments
        if 'gdp_growth' in scenario:
            model.economic_assumptions['gdp_growth_rate'] = scenario['gdp_growth']
        if 'inflation' in scenario:
            model.economic_assumptions['inflation_rate'] = scenario['inflation']
        
        # Run projection
        projection = model.project_outlook(2024, 2024 + years)
        
        results.append({
            'scenario': scenario['name'],
            'final_debt': projection['total_debt'][-1],
            'final_deficit': projection['annual_deficit'][-1],
            'debt_to_gdp': projection.get('debt_to_gdp_ratio', [0])[-1]
        })
    
    return results

# Define scenarios
scenarios = [
    {'name': 'Baseline', 'gdp_growth': 0.023},
    {'name': 'Strong Growth', 'gdp_growth': 0.035},
    {'name': 'Recession', 'gdp_growth': 0.005},
    {'name': 'High Inflation', 'gdp_growth': 0.023, 'inflation': 0.045},
]

# Run batch
batch_results = batch_simulate(scenarios)

print("\n📊 Batch Results:")
print("-" * 60)
print(f"{'Scenario':<20} {'Final Debt':>15} {'Final Deficit':>15}")
print("-" * 60)
for r in batch_results:
    print(f"{r['scenario']:<20} ${r['final_debt']/1e12:>11.2f}T ${r['final_deficit']/1e9:>11.0f}B")

---

## 4. Integration Patterns

### Pattern 1: Web Application Backend

```python
from flask import Flask, jsonify
from core.simulation import run_comprehensive_simulation

app = Flask(__name__)

@app.route('/api/simulate', methods=['POST'])
def simulate():
    config = request.json
    results = run_comprehensive_simulation(**config)
    return jsonify(results)
```

### Pattern 2: Data Pipeline

```python
import pandas as pd
from core.combined_outlook import CombinedFiscalOutlookModel

# Generate dataset
model = CombinedFiscalOutlookModel()
results = model.project_outlook(2024, 2054)
df = pd.DataFrame(results)
df.to_csv('fiscal_projections.csv')
```

### Pattern 3: Automated Reporting

```python
from core.report_generator import generate_report

# Generate PDF report
report = generate_report(
    policy='M4A',
    years=10,
    format='pdf'
)
```

In [ ]:
# Example: Export results to DataFrame
import pandas as pd

# Run a projection
model = CombinedFiscalOutlookModel()
projection = model.project_outlook(2024, 2034)

# Convert to DataFrame
years = list(range(2024, 2035))
df = pd.DataFrame({
    'Year': years,
    'Total Debt ($T)': [d/1e12 for d in projection['total_debt']],
    'Annual Deficit ($B)': [d/1e9 for d in projection['annual_deficit']],
    'Revenue ($T)': [r/1e12 for r in projection['total_revenue']],
})

print("📊 Projection DataFrame:")
print(df.to_string(index=False))

# Export
# df.to_csv('projection_export.csv', index=False)
# df.to_excel('projection_export.xlsx', index=False)
print("\n💡 Uncomment above lines to export to CSV/Excel")

---

## 🎓 What You've Learned

✅ PoliSim provides REST API and direct import modes  
✅ API client class simplifies programmatic access  
✅ Batch processing enables scenario comparison  
✅ Results export to DataFrames for analysis  

---

## 📚 Next Steps

| Notebook | Topic | Time |
|----------|-------|------|
| **09** | Custom Policy Design | 60-90 min |
| **10** | Capstone Analysis | 2-3 hours |

---

*Continue to Notebook 09: Custom Policies* →